In [1]:
import tensorflow as tf
import cv2
import matplotlib.pyplot as plt
import numpy as np
import os
import random
import pandas as pd
from skimage import io
from shutil import copyfile
import sys
import time
from keras.preprocessing.image import load_img, img_to_array
!pip install wget

  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9675 sha256=5831472f6554e557a991821cda71a7df80963315d8f999de645f19adcb43d12c
  Stored in directory: /root/.cache/pip/wheels/a1/b6/7c/0e63e34eb06634181c63adacca38b79ff8f35c37e3c13e3c02
Successfully built wget


In [2]:
import wget
url = 'https://storage.googleapis.com/openimages/2018_04/train/train-annotations-bbox.csv'
filename = wget.download(url)

In [3]:
url = 'https://storage.googleapis.com/openimages/2018_04/train/train-images-boxable-with-rotation.csv'
filename = wget.download(url)

In [4]:
url = 'https://storage.googleapis.com/openimages/v5/class-descriptions-boxable.csv'
filename = wget.download(url)

In [5]:
images_boxable_file = 'train-images-boxable-with-rotation.csv'
annotations_bbox_file = 'train-annotations-bbox.csv'
class_descriptions_file = 'class-descriptions-boxable.csv'

In [6]:
images_boxable = pd.read_csv(images_boxable_file)
images_boxable.head(3)

,ImageID,Subset,OriginalURL,OriginalLandingURL,License,AuthorProfileURL,Author,Title,OriginalSize,OriginalMD5,Thumbnail300KURL,Rotation
0,4fa8054781a4c382,train,https://farm3.staticflickr.com/5310/5898076654...,https://www.flickr.com/photos/michael-beat/589...,https://creativecommons.org/licenses/by/2.0/,https://www.flickr.com/people/michael-beat/,Michael Beat,...die FNF-Kerze,4405052,KFukvivpCM5QXl5SqKe41g==,https://c1.staticflickr.com/6/5310/5898076654_...,0.0
1,b37f763ae67d0888,train,https://c1.staticflickr.com/1/67/197493648_628...,https://www.flickr.com/photos/drstarbuck/19749...,https://creativecommons.org/licenses/by/2.0/,https://www.flickr.com/people/drstarbuck/,Karen,Three boys on a hill,494555,9IzEn38GRNsVpATuv7gzEA==,https://c3.staticflickr.com/1/67/197493648_628...,0.0
2,7e8584b0f487cb9e,train,https://c7.staticflickr.com/8/7056/7143870979_...,https://www.flickr.com/photos/circasassy/71438...,https://creativecommons.org/licenses/by/2.0/,https://www.flickr.com/people/circasassy/,CircaSassy,A Christmas carol and The cricket on the heart...,2371584,3hQwu0iSzY1VIoXiwp0/Mg==,https://c7.staticflickr.com/8/7056/7143870979_...,0.0


In [7]:
annotations_bbox = pd.read_csv(annotations_bbox_file)
annotations_bbox.head()

,ImageID,Source,LabelName,Confidence,XMin,XMax,YMin,YMax,IsOccluded,IsTruncated,IsGroupOf,IsDepiction,IsInside
0,000002b66c9c498e,xclick,/m/01g317,1,0.012500,0.195312,0.148438,0.587500,0,1,0,0,0
1,000002b66c9c498e,xclick,/m/01g317,1,0.025000,0.276563,0.714063,0.948438,0,1,0,0,0
2,000002b66c9c498e,xclick,/m/01g317,1,0.151562,0.310937,0.198437,0.590625,1,0,0,0,0
3,000002b66c9c498e,xclick,/m/01g317,1,0.256250,0.429688,0.651563,0.925000,1,0,0,0,0
4,000002b66c9c498e,xclick,/m/01g317,1,0.257812,0.346875,0.235938,0.385938,1,0,0,0,0


In [8]:
class_descriptions = pd.read_csv(class_descriptions_file, header=None)
class_descriptions.head()

,0,1
0,/m/011k07,Tortoise
1,/m/011q46kg,Container
2,/m/012074,Magpie
3,/m/0120dh,Sea turtle
4,/m/01226z,Football


In [9]:
def plot_bbox(img_id):
  img_url = images_boxable.loc[images_boxable["ImageID"]==img_id]['OriginalURL'].values[0]
  img = io.imread(img_url)
  height, width, channel = img.shape
  print(f"Image: {img.shape}")
  bboxs = annotations_bbox[annotations_bbox['ImageID']==img_id]
  for index, row in bboxs.iterrows():
      xmin = row['XMin']
      xmax = row['XMax']
      ymin = row['YMin']
      ymax = row['YMax']
      xmin = int(xmin*width)
      xmax = int(xmax*width)
      ymin = int(ymin*height)
      ymax = int(ymax*height)
      label_name = row['LabelName']
      class_series = class_descriptions[class_descriptions[0]==label_name]
      class_name = class_series[1].values[0]
      print(f"Coordinates: {xmin,ymin}, {xmax,ymax}")
      cv2.rectangle(img, (xmin,ymin), (xmax,ymax), (255,0,0), 5)
      font = cv2.FONT_HERSHEY_SIMPLEX
      cv2.putText(img, class_name, (xmin,ymin-10), font, 3, (0,255,0), 5)
  plt.figure(figsize=(15,10))
  plt.title('Image with Bounding Box')
  plt.imshow(img)
  plt.axis('off')
  plt.show()

In [10]:
#Creating random sample list for display
random_img_ids = annotations_bbox["ImageID"].value_counts().tail(50).index.values
for img_id in random.sample(list(random_img_ids), 3):
  plot_bbox(img_id)

Output hidden; open in https://colab.research.google.com to view.

In [11]:
table = class_descriptions[class_descriptions[1]=='Table'] 
chair = class_descriptions[class_descriptions[1]=='Chair']
person = class_descriptions[class_descriptions[1]=='Person']
laptop = class_descriptions[class_descriptions[1]=='Laptop']

In [16]:
label_table = table[0].values[0]
label_chair = chair[0].values[0]
label_person = person[0].values[0]
label_laptop = laptop[0].values[0]
label_laptop

'/m/01c648'

In [19]:
bbox_table = annotations_bbox[annotations_bbox["LabelName"]==label_table]
bbox_chair = annotations_bbox[annotations_bbox["LabelName"]==label_chair]
bbox_person = annotations_bbox[annotations_bbox["LabelName"]==label_person]
bbox_laptop = annotations_bbox[annotations_bbox["LabelName"]==label_laptop]

In [21]:
bbox_laptop.head()

,ImageID,Source,LabelName,Confidence,XMin,XMax,YMin,YMax,IsOccluded,IsTruncated,IsGroupOf,IsDepiction,IsInside
583,000060e3121c7305,xclick,/m/01c648,1,0.322500,0.389375,0.580131,0.669166,0,0,0,0,0
3539,00022a6311159428,xclick,/m/01c648,1,0.126875,0.484375,0.638281,0.871094,0,0,0,0,0
4018,0002af997ecdfcf4,xclick,/m/01c648,1,0.250000,0.363750,0.236667,0.334444,0,0,0,0,0
5473,000396ae942e8778,xclick,/m/01c648,1,0.000000,0.995000,0.000000,0.996390,0,0,0,0,0
6449,00048edce2a37368,xclick,/m/01c648,1,0.000000,0.225000,0.710000,0.999167,1,1,0,0,0


In [22]:
print(len(bbox_table))
print(len(bbox_chair))
print(len(bbox_person))
print(len(bbox_laptop))

85691
132483
1034721
9327


In [23]:
table_img_id = bbox_table['ImageID']
chair_img_id = bbox_chair['ImageID']
person_img_id = bbox_person['ImageID']
laptop_img_id = bbox_laptop['ImageID']

In [25]:
n=50
subset_table_id = random.sample(list(table_img_id), n)
subset_chair_id = random.sample(list(chair_img_id), n)
subset_person_id = random.sample(list(person_img_id), n)
subset_laptop_id = random.sample(list(laptop_img_id), n)

In [27]:
subtable_pd = images_boxable.loc[images_boxable["ImageID"].isin(subset_table_id)]
subchair_pd = images_boxable.loc[images_boxable["ImageID"].isin(subset_chair_id)]
subperson_pd = images_boxable.loc[images_boxable["ImageID"].isin(subset_person_id)]
sublaptop_pd = images_boxable.loc[images_boxable["ImageID"].isin(subset_laptop_id)]
sublaptop_pd.head()

,ImageID,Subset,OriginalURL,OriginalLandingURL,License,AuthorProfileURL,Author,Title,OriginalSize,OriginalMD5,Thumbnail300KURL,Rotation
13950,45fe56d21ecb4c12,train,https://farm3.staticflickr.com/7014/6516273755...,https://www.flickr.com/photos/hightechdad/6516...,https://creativecommons.org/licenses/by/2.0/,https://www.flickr.com/people/hightechdad/,Michael Sheehan,The ASUS Zenbook UX31E Ultrabook,319084,eRmdjkRE0+uTLBpd6NVzlg==,https://c4.staticflickr.com/8/7014/6516273755_...,0.0
40757,9e6757a74dd68089,train,https://c2.staticflickr.com/1/188/462654682_ca...,https://www.flickr.com/photos/raaphorst/462654682,https://creativecommons.org/licenses/by/2.0/,https://www.flickr.com/people/raaphorst/,Marco Raaphorst,Aad van Nieuwkerk,2718042,MW4poRpSg+XRJwMvSWE/Qg==,https://c3.staticflickr.com/1/188/462654682_07...,0.0
65677,beafab65c2babcf5,train,https://farm6.staticflickr.com/292/18885618134...,https://www.flickr.com/photos/nzdefenceforce/1...,https://creativecommons.org/licenses/by/2.0/,https://www.flickr.com/people/nzdefenceforce/,New Zealand Defence Force,-,325659,YmxttS0bNnM8k07jv0nLHQ==,https://c5.staticflickr.com/1/292/18885618134_...,0.0
93212,e3057724886cd615,train,https://farm6.staticflickr.com/8029/8051976005...,https://www.flickr.com/photos/giorgiop5/805197...,https://creativecommons.org/licenses/by/2.0/,https://www.flickr.com/people/giorgiop5/,giorgiop5,Make Opendata September 2012,3473630,vqKbf266Lue16DgLpT//6Q==,https://c6.staticflickr.com/9/8029/8051976005_...,0.0
103450,ee0b067ff0d75a6d,train,https://c6.staticflickr.com/8/7162/6449916337_...,https://www.flickr.com/photos/draket/6449916337,https://creativecommons.org/licenses/by/2.0/,https://www.flickr.com/people/draket/,Ted Drake,Chach Sikes,2508916,7ox9JnGEJ5nIGtwFlR8kZw==,https://c5.staticflickr.com/8/7162/6449916337_...,NaN


In [31]:
#Setting the index by .set_index and converting Pandas DF to dict by .to_dict()
subtable_dict = subtable_pd[["ImageID", "OriginalURL"]].set_index('ImageID')["OriginalURL"].to_dict()
subchair_dict = subchair_pd[["ImageID", "OriginalURL"]].set_index('ImageID')["OriginalURL"].to_dict()
subperson_dict = subperson_pd[["ImageID", "OriginalURL"]].set_index('ImageID')["OriginalURL"].to_dict()
sublaptop_dict = sublaptop_pd[["ImageID", "OriginalURL"]].set_index('ImageID')["OriginalURL"].to_dict()

In [33]:
mappings = [subtable_dict, subchair_dict, subperson_dict, sublaptop_dict]
classes = ['Table', 'Chair', 'Person', 'Laptop']

In [34]:
for idx, obj_type in enumerate(classes):
  issues = 0
  if not os.path.exists(obj_type):
    os.mkdir(obj_type)
  for img_id , url in mappings[idx].items():
    try:
      img = io.imread(url)
      saved_path = os.path.join(obj_type, img_id+".jpg")
      io.imsave(saved_path, img)
    except Exception as e:
      issues = issues + 1
  print(f'Image issues = {issues}')

Image issues = 5
Image issues = 5
Image issues = 2
Image issues = 6
